In [3]:
import os, time, datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx
import minorminer

from dwave.system import DWaveCliqueSampler, DWaveSampler

# -----------------------------
# Config (set BEFORE any function)
# -----------------------------
ENDPOINT = os.getenv("DWAVE_API_ENDPOINT", "https://cloud.dwavesys.com/sapi")
TOKEN    = os.getenv("DWAVE_API_TOKEN", None)  # export DWAVE_API_TOKEN=...
SOLVER   = os.getenv("DWAVE_SOLVER", "Advantage2_system1.6")

# tqdm (fallback if not installed)
try:
    from tqdm.auto import tqdm
except Exception:
    def tqdm(x, **kwargs):  # no-op fallback
        return x

# -----------------------------
# Logging
# -----------------------------
def log(msg):
    ts = datetime.datetime.now().strftime("%H:%M:%S")
    print(f"[{ts}] {msg}", flush=True)

# -----------------------------
# QPU helpers
# -----------------------------
def get_qpu(solver=None):
    solver = solver or SOLVER
    if TOKEN is None:
        log("WARNING: DWAVE_API_TOKEN not set; DWaveSampler may fail to auth.")
    return DWaveSampler(endpoint=ENDPOINT, token=TOKEN, solver=solver)

def get_L_list_from_clique(L_min=8, step=5, solver=None):
    """Use DWaveCliqueSampler to discover largest clique size (no credits)."""
    solver = solver or SOLVER
    if TOKEN is None:
        log("WARNING: DWAVE_API_TOKEN not set; CliqueSampler may fail to auth.")
    cs = DWaveCliqueSampler(endpoint=ENDPOINT, token=TOKEN, solver=solver)
    L_max = getattr(cs, "largest_clique_size", None)
    if not isinstance(L_max, int) or L_max <= 0:
        raise RuntimeError("largest_clique_size. Check the Solver.")
    if L_min > L_max:
        L_min = L_max
    L_list = list(range(L_min, L_max + 1, step)) or [L_max]
    log(f"[L_max] {L_max}  |  [L_list] {L_list} (step={step})")
    return L_list, L_max

# -----------------------------
# Embedding + chain length
# -----------------------------
def find_clique_embedding(qpu, L, seed=0, verbose=True):
    """Manual K_L embedding on the hardware graph using minorminer."""
    hw = qpu.to_networkx_graph()
    K = nx.complete_graph(L)
    emb = minorminer.find_embedding(K.edges(), hw.edges(), random_seed=seed)
    if not emb:
        raise RuntimeError(f"Embedding failed for K_{L}")
    if verbose:
        lens = [len(c) for c in emb.values()]
        log(f"[Embedding-KL] L={L} chains={len(emb)} meanCL={np.mean(lens):.2f} maxCL={np.max(lens)}")
    return emb

def chain_lengths_from_embedding(emb):
    return np.array([len(c) for c in emb.values()], dtype=int)

def mean_chainlen_via_clique(qpu, L, seed=0, verbose=True):
    """Embed K_L and return (mean_len, max_len, lens-array)."""
    emb  = find_clique_embedding(qpu, L, seed=seed, verbose=False)
    lens = chain_lengths_from_embedding(emb)
    mean_len = float(np.mean(lens)) if len(lens) else float('nan')
    max_len  = int(np.max(lens))    if len(lens) else -1
    if verbose:
        log(f"[ChainLen] L={L} meanCL={mean_len:.3f} maxCL={max_len} (chains={len(lens)})")
    return mean_len, max_len, lens

# -----------------------------
# Build curve + save CSV/figure
# -----------------------------
def build_chainlen_vs_L(L_list, *, emb_seed_base=0,
                        out_dir="qa_chainlen_only",
                        save_fig="Img/chainlen_scaling_all_cropped.pdf",
                        verbose=True, show_progress=True):
    """Measure mean chain length vs L using clique embeddings only."""
    os.makedirs(out_dir, exist_ok=True)
    fig_dir = os.path.dirname(save_fig)
    if fig_dir:
        os.makedirs(fig_dir, exist_ok=True)

    qpu = get_qpu()
    rows = []

    it = tqdm(L_list, disable=not show_progress, desc="Clique-embedding (length only)")
    for L in it:
        mean_len, max_len, lens = mean_chainlen_via_clique(qpu, L, seed=emb_seed_base+L, verbose=verbose)
        rows.append(dict(L=int(L), emb_seed=int(emb_seed_base+L),
                         mean_chainlen=mean_len, max_chainlen=max_len, n_chains=len(lens)))

    df = pd.DataFrame(rows).sort_values("L").reset_index(drop=True)
    ts = int(time.time())
    csv_path = os.path.join(out_dir, f"chainlen_vs_L_{ts}.csv")
    df.to_csv(csv_path, index=False); log(f"[saved] {csv_path}")

    # ---- Linear regression (numpy) ----
    X = df["L"].values.astype(float)
    y = df["mean_chainlen"].values.astype(float)
    if len(X) >= 2:
        a, b = np.polyfit(X, y, 1)  # slope, intercept
        y_pred = a * X + b
        ss_res = float(np.sum((y - y_pred) ** 2))
        ss_tot = float(np.sum((y - np.mean(y)) ** 2))
        r2 = 1.0 - ss_res / ss_tot if ss_tot > 0 else 1.0
    else:
        a, b, r2 = float("nan"), float("nan"), float("nan")
        y_pred = y

    # ---- Plot (legend shows observed & fit with slope/R^2) ----
    plt.rcParams.update({
        "font.size": 16, "axes.labelsize": 18, "axes.titlesize": 18,
        "xtick.labelsize": 16, "ytick.labelsize": 16,
    })
    plt.figure(figsize=(8, 6))
    plt.plot(df["L"], df["mean_chainlen"], marker="o", linestyle="-", linewidth=2, label="Observed mean")
    plt.plot(df["L"], y_pred, linestyle="--", linewidth=2, label=f"fit: slope={a:.3f}, $R^2$={r2:.3f}")
    plt.xlabel("L (logical variables)")
    plt.ylabel("Mean chain length")
    plt.grid(True, linestyle="--", alpha=0.6)
    plt.legend()
    plt.tight_layout()
    #plt.savefig(save_fig, bbox_inches="tight", dpi=160); log(f"[saved] {save_fig}")
    #plt.close()

    return dict(df=df, csv_path=csv_path, fig_path=save_fig, slope=a, intercept=b, r2=r2)

# -----------------------------
# Main
# -----------------------------
if __name__ == "__main__":
    # 0) L candidates by clique size (no credits)
    L_list, L_max = get_L_list_from_clique(L_min=5, step=5, solver=SOLVER)

    # 1) Measure only chain length vs L (no sampling)
    res = build_chainlen_vs_L(
        L_list,
        emb_seed_base=0,
        out_dir="qa_chainlen_only",
        #save_fig="Img/chainlen_scaling_all_cropped.pdf",
        verbose=True,
        show_progress=True,
    )
    log(f"[fit] slope={res['slope']:.4f}, R^2={res['r2']:.4f}")


[19:02:39] WARNING: DWAVE_API_TOKEN not set; CliqueSampler may fail to auth.
[19:02:41] [L_max] 104  |  [L_list] [5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80, 85, 90, 95, 100] (step=5)
[19:02:41] WARNING: DWAVE_API_TOKEN not set; DWaveSampler may fail to auth.


Clique-embedding (length only):   0%|                                                           | 0/20 [00:00<?, ?it/s]

[19:02:42] [ChainLen] L=5 meanCL=1.200 maxCL=2 (chains=5)
[19:02:42] [ChainLen] L=10 meanCL=1.700 maxCL=2 (chains=10)


Clique-embedding (length only):  10%|█████                                              | 2/20 [00:00<00:01, 12.21it/s]

[19:02:43] [ChainLen] L=15 meanCL=2.000 maxCL=3 (chains=15)
[19:02:43] [ChainLen] L=20 meanCL=2.500 maxCL=3 (chains=20)


...

Clique-embedding (length only):  95%|███████████████████████████████████████████████▌  | 19/20 [04:30<00:35, 35.94s/it]

[19:08:04] [ChainLen] L=100 meanCL=14.500 maxCL=25 (chains=100)


Clique-embedding (length only): 100%|██████████████████████████████████████████████████| 20/20 [05:21<00:00, 16.06s/it]

[19:08:04] [saved] qa_chainlen_only\chainlen_vs_L_1759486084.csv


[19:08:04] [saved] Img/chainlen_scaling_all_cropped.pdf
[19:08:04] [fit] slope=0.1221, R^2=0.9683
